# Langchain RAG notebook

## The basics

In [ ]:
# Initialize requirements
from dotenv import load_dotenv
import os
import getpass

from langchain_anthropic import ChatAnthropic
from langchain_openai import ChatOpenAI

from langchain_core.messages import HumanMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate

from langchain_core.output_parsers import StrOutputParser

load_dotenv()

In [12]:
# Loading models
gpt_mini = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv('OPENAI_API_KEY'))
claude_sonnet = ChatAnthropic(model="claude-3-sonnet-20240229")

In [25]:
# Writing prompts
prompt1 = ChatPromptTemplate.from_messages([
    ("system", "You are a world class technical documentation writer. Keep all responses to one sentence max"),
    ("user", "{input}")])

prompt2 = [
    SystemMessage(content="Translate the following from English into Italian"),
    HumanMessage(content="hi!"),
]

In [26]:
output_parser = StrOutputParser()

In [21]:

gpt_mini.invoke(prompt2)

AIMessage(content='Ciao!', response_metadata={'token_usage': {'completion_tokens': 3, 'prompt_tokens': 20, 'total_tokens': 23}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_8b761cb050', 'finish_reason': 'stop', 'logprobs': None}, id='run-02346bbb-48f2-4a99-8649-3cf9e67d85d9-0', usage_metadata={'input_tokens': 20, 'output_tokens': 3, 'total_tokens': 23})

In [27]:
chain = prompt1 | gpt_mini | output_parser

In [28]:
chain.invoke({"input": "How to install a new hard drive on a computer"})

'Power off the computer, open the case, connect the hard drive to the motherboard and power supply, secure it in place, and then close the case.'

## RAG on Opentrons documentation

For langchain docs see: https://python.langchain.com/v0.2/docs/tutorials/rag/

Basic workflow
* Indexing - convert input data into a format that is easy to search
* Retreival - return relevant documents or section of test given an input query
* Generate - Pass prompt with retrieved information to model and generate response

Concepts
* Vector store - You convert raw input data into embedding vectors that a model can than search with the embedded query
* Embedding - Embedding models create a vector representation of your piece of text. Think of it as a vector of number that captures the semantic meaning of text.


### Setup

In [68]:
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.documents import Document
from langchain_core.runnables import RunnablePassthrough

from langchain_openai import ChatOpenAI
from langchain_openai import OpenAIEmbeddings

import os
import re
from dotenv import load_dotenv

load_dotenv()

True

In [36]:
gpt_mini = ChatOpenAI(model="gpt-4o-mini", api_key=os.getenv('OPENAI_API_KEY'))

### Load docs from PDF
See https://api.python.langchain.com/en/latest/document_loaders/langchain_community.document_loaders.pdf.PyPDFLoader.html

Loader chunks by page and stores page numbers in metadata.

In [40]:
loader = PyPDFLoader("rag-documentation/opentrons-docs.pdf")
documents = loader.load()

The resulting PDF text has new line characters inserted after essentially every word. Post-processing to clean up the text.

In [80]:
def clean_text(text):
    # Remove single newlines
    text = re.sub(r'(?<!\n)\n(?!\n)', ' ', text)
    # Replace multiple newlines with a single newline
    text = re.sub(r'\n+', '\n', text)
    return text.strip()

cleaned_docs = [Document(page_content=clean_text(doc.page_content), metadata=doc.metadata) for doc in documents]

print(cleaned_docs[15].page_content)

Temperature profiles only control the temperature of the block in the Thermocycler. You should set a lid temperature before executing the profile using set_lid_temperature(). Deck slots Deck slots are where you place hardware items on the deck surface of your Opentrons robot. In the API, you load the corresponding items into your protocol with methods like ProtocolContext.load_labware, ProtocolContext.load_module, or ProtocolContext.load_trash_bin. When you call these methods, you need to specify which slot to load the item in. Physical Deck Labels Flex uses a coordinate labeling system for slots A1 (back left) through D4 (front right). Columns 1 through 3 are in the working area and are accessible by pipettes and the gripper. Column 4 is in the staging area and is only accessible by the gripper. For more information on staging area slots, see Deck Configuration below. _images/flex-deck.svg OT-2 uses a numeric labeling system for slots 1 (front left) through 11 (back center). The back 

### Create a text splitter
https://api.python.langchain.com/en/latest/character/langchain_text_splitters.character.RecursiveCharacterTextSplitter.html

The text splitter takes the document and splits it into further chunks, smaller than the page chunks returned by `PyPDFLoader`.

The `RecursiveCharacterTextSplitter` works by trying to split on a list of characters in order. It starts with the first character (often newlines), and if the resulting chunks are too large, it moves on to the next character (often periods), and so on. This "recursive" approach allows it to intelligently split text while trying to maintain semantic coherence as much as possible.

In [81]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(cleaned_docs)

In [43]:
splits

[Document(metadata={'source': 'rag-documentation/opentrons-docs.pdf', 'page': 0}, page_content='Introduction\nto\nOpentrons\nOT-2\nfor\nAI\nAssistants\nThe\nOpentrons\nOT-2\nis\nan\nopen-source\nliquid\nhandling\nrobot\ndesigned\nfor\nautomated\npipetting\nand\nprotocol\nexecution\nin\nlife\nscience\nlaboratories.\nThis\ndocument\nprovides\na\nconcise\noverview\nof\nthe\nkey\ninformation\nneeded\nto\ncontrol\nan\nOT-2\nrobot\nusing\nan\nAI\nlanguage\nmodel.\nThe\ngoal\nis\nto\nenable\nAI\nassistants\nto\nunderstand\nOT-2\ncapabilities,\nconstraints,\nand\nAPI\ncommands\nin\norder\nto\ngenerate\nvalid\nOT-2\nprotocols\nbased\non\nhigh-level\nuser\nintent.\nThe\ndocument\ncovers:\nOT-2\nhardware\noverview\nand\nspecifications\n●\nAPI\nbasics\nand\nkey\nconcepts\n●\nPipettes\nand\nliquid\nhandling\n●\nLabware\nand\ndeck\nsetup\n●\nBuilding\nblock\ncommands\nfor\naspirate,\ndispense,\netc.\n●\nComplex\ncommands\nfor\ncommon\npipetting\npatterns\n●\nProtocol\nexamples\ndemonstrating\nAPI\nu

### Create embeddings and store in Chroma

Chroma: https://docs.trychroma.com/getting-started

Chroma is a vector database that stores embedding vectors.

In [82]:
embeddings = OpenAIEmbeddings()
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

### Set up the retriever
https://python.langchain.com/v0.1/docs/modules/data_connection/retrievers/

Retrievers accept a string query as input and return a list of Document's (or splits of documents) as output.

`k` determines the number of top results to return from the similarity search. Setting `k = 3` means that for each query, the retriever will return the 3 most similar documents (or chunks) from the vector store.

In [83]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

### Create custom prompt template

`prompt` is a step in the langchain pipeline. In this case it's automatically unpacking a dict passed to it with entries `context` and `question`.

In [84]:
prompt = ChatPromptTemplate.from_messages([
    ("system", "You are an AI assistant specialized in generating Python code to control Opentrons liquid handling robots. Use the following context from the Opentrons documentation to help you generate accurate and safe code."),
    ("human", "Context: {context}\n\nHuman: {question}\n\nGenerate Python code to accomplish this task using the Opentrons API:"),
])

### Create RAG chain

In [94]:
question = "How do I set the temperature of a thermocycler block on an Opentrons robot?"

docs = retriever.get_relevant_documents(question)

def format_docs(docs):
    return "".join(doc.page_content for doc in docs)

context = format_docs(docs)  # to be passed to model
prompt_input = {"context": context, "question": question}

prompt_input

{'context': 'The Thermocycler can control its block temperature, including holding at a temperature and adjusting for the volume of liquid held in its loaded plate. Temperature To set the block temperature inside the Thermocycler, use set_block_temperature(). At minimum you have to specify a temperature in degrees Celsius: tc_mod.set_block_temperature(temperature=4) If you don’t specify any other parameters, the Thermocycler will hold this temperature until a new temperature is set, deactivate_block() is called, or the module is powered off. New in version 2.0. Hold Time You can optionally instruct the Thermocycler to hold its block temperature for a specific amount of time. You can specify hold_time_minutes, hold_time_seconds, or both (in which case they will be added together). For example, this will set the block to 4 °C for 4 minutes and 15 seconds: tc_mod.set_block_temperature( temperature=4, hold_time_minutes=4, hold_time_seconds=15) Note Your protocol will not proceed to further

In [97]:
prompted_input = prompt.format(**prompt_input)
prompted_input

'System: You are an AI assistant specialized in generating Python code to control Opentrons liquid handling robots. Use the following context from the Opentrons documentation to help you generate accurate and safe code.\nHuman: Context: The Thermocycler can control its block temperature, including holding at a temperature and adjusting for the volume of liquid held in its loaded plate. Temperature To set the block temperature inside the Thermocycler, use set_block_temperature(). At minimum you have to specify a temperature in degrees Celsius: tc_mod.set_block_temperature(temperature=4) If you don’t specify any other parameters, the Thermocycler will hold this temperature until a new temperature is set, deactivate_block() is called, or the module is powered off. New in version 2.0. Hold Time You can optionally instruct the Thermocycler to hold its block temperature for a specific amount of time. You can specify hold_time_minutes, hold_time_seconds, or both (in which case they will be ad

In [99]:
model_output = gpt_mini.invoke(prompted_input)

In [104]:
final_output = StrOutputParser().parse(model_output)

print(final_output.content)

To set the temperature of a thermocycler block on an Opentrons robot, you can use the `set_block_temperature()` method from the Thermocycler module. Below is an example of Python code that sets the thermocycler block to 4 °C for 4 minutes and 15 seconds:

```python
from opentrons import protocol_api

metadata = {
    'apiLevel': '2.0',
}

def run(protocol: protocol_api.ProtocolContext):
    # Load the Thermocycler module
    tc_mod = protocol.load_module('thermocycler', '1')  # Adjust the slot as necessary

    # Set the block temperature to 4 °C and hold for 4 minutes and 15 seconds
    tc_mod.set_block_temperature(temperature=4, hold_time_minutes=4, hold_time_seconds=15)

    # Optionally, you can set the lid temperature if needed
    # tc_mod.set_lid_temperature(temperature=100)  # Example to set lid temperature to 100 °C

    # Add any other protocol steps below
```

### Explanation:
- This code first imports the necessary modules from the Opentrons API.
- It defines a `run` functi

In [105]:
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | gpt_mini
    | StrOutputParser()
)

In [108]:
response = rag_chain.invoke("Write code to pipette 100 uL from well A1 to well B1 on a 96-well plate")
print(response)

Here’s a Python script using the Opentrons API to pipette 100 µL from well A1 to well B1 on a 96-well plate. This code assumes you have the appropriate labware and pipette loaded as described in your context:

```python
from opentrons import protocol_api

# metadata
metadata = {
    'apiLevel': '2.17',
    'protocolName': 'Pipetting from A1 to B1',
    'author': 'Your Name',
    'description': 'Transfer 100 µL from well A1 to well B1 on a 96-well plate',
}

def run(protocol: protocol_api.ProtocolContext):
    # Load a tip rack for 200 µL tips
    tiprack = protocol.load_labware('opentrons_flex_96_tiprack_200ul', 'D1')

    # Load a 96-well plate
    well_plate = protocol.load_labware('nest_96_wellplate_200ul_flat', 'A1')

    # Load a single-channel pipette on the left mount
    pipette = protocol.load_instrument('p300_single', 'left', tip_racks=[tiprack])

    # Pick up a tip
    pipette.pick_up_tip()

    # Aspirate 100 µL from well A1
    pipette.aspirate(100, well_plate['A1'])

   